In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import time
import asyncio
import aiohttp
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('api_key')

request_header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key
}

base_url = "https://kr.api.riotgames.com/tft/"
match_base_url = "https://asia.api.riotgames.com/tft/match/v1/"

In [2]:
# 챌린저 유저 데이터 가져오는 함수
async def fetch_challenger_data(session: aiohttp.ClientSession):
    url = base_url + 'league/v1/challenger'
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            return await response.json()

In [3]:
# 매치 아이디 가져오는 함수수
async def fetch_match_ids(session: aiohttp.ClientSession, puuid: str, count: int = 20):
    url = f"{match_base_url}matches/by-puuid/{puuid}/ids?count={count}"
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            if response.status == 200:
                match_ids = await response.json()
                return {'puuid': puuid, 'match_ids': match_ids}

In [4]:
# 매치 상세 데이터 가져오는 함수
async def fetch_match_detail(session: aiohttp.ClientSession, match_id: str):
    url = f"{match_base_url}matches/{match_id}"
    while True:
        async with session.get(url, headers=request_header) as response:
            if response.status == 429:
                retry_after = int(response.headers.get('Retry-After', 1))
                await asyncio.sleep(retry_after)
                continue
            return await response.json()

In [5]:
# 챌린저 데이터 DataFrame 생성
async def process_challenger_data():
    async with aiohttp.ClientSession() as session:
        challenger  = await fetch_challenger_data(session)
        challenger_df = pd.DataFrame(challenger['entries'])
        challenger_df.to_csv('challenger.csv', index=False)
        return challenger_df

In [6]:
# 매치 아이디 DataFrame 생성
async def process_match_ids():
    async with aiohttp.ClientSession() as session:
        challenger_df = await process_challenger_data()
        challenger_puuid = challenger_df['puuid'].tolist()
        match_data = []
        for puuid in challenger_puuid:
            match_ids = await fetch_match_ids(session, puuid)
            match_data.append(match_ids)
            await asyncio.sleep(1)
        match_data_df = pd.DataFrame(match_data)
        match_data_df.to_csv('challenger_match_ids.csv', index=False)
        return match_data_df

In [7]:
async def main():
    start_time = time.time()
    challenger_df = await process_challenger_data()
    challenger_df.to_csv('challenger_user.csv', index=False)
    print(f"처리 시간: {time.time() - start_time:.2f}초")
    print(challenger_df)

In [8]:
async def main():
    challenger_df = await process_challenger_data()
    print(challenger_df)
    match_ids_df = await process_match_ids()
    print(match_ids_df)

await main()

                                            summonerId  \
0      dHC7Lj6ua-tf1_KAkv0OTXNGNUcwSXCVKi0COVTkSW1Eyno   
1      d_L7oC_ZLj2XTtx2l83NTZcfsNyMZbmvrmEgvInhW5b6p40   
2    _w17WMTe0OnGCRP3E5GmDtLwLuuPWjFa4AdjMju9rpJ6va...   
3      rCboap1-tmroGQdxNfxR0ifh1KyrJ0lpzppPraO7TCi5o0Q   
4      AIJes1lZD8ycle3VVWhH0t8ZtYTB9HqI9jGtPExcR50SOhg   
..                                                 ...   
295    SERFONLIrMKHYOmgAifngLh2_dLHUSk8QtzbpbcBEGfY5c8   
296  6umbsrTWP8_eiIwGm90Mt20WoGTBUAWs-fCrJ_WvYmSe5K...   
297    7Qk51zoj2RG-sYTleflFiDzOooNfJQ8UBqBmmRkIujnQigQ   
298   KqC-TRVchC9XXIBpyB3pTeeJbiz8PORYdxxsb6Wc6jQInbIA   
299     ROl4BUpRg25LfFPGIBn7z5tz90MPrdaIBaDXcbe2Hm0Qfw   

                                                 puuid  leaguePoints rank  \
0    fS2PZn5F89qCGQCUW_vDBw5WPCiblNvA9VDvuPts3zeHPi...          1760    I   
1    I9oeXQ-436Y7bT5qgUbodeTFzcQ1-Ln-FnCCwq2LmKK1qd...          1739    I   
2    GnJrE9eq5jOPNWkP_bFqRx3I9jA8mTbGqT86GmFRp9Nqzq...          1725    

In [18]:
# 챌린저 유저 데이터 저장
import pymysql

conn = pymysql.connect(
    host='localhost',
    user='root',
    password='Chokh091002!',
    db='tft',
    charset='utf8'
)

cursor = conn.cursor(pymysql.cursors.DictCursor)

query = "create table if not exists challenger_user (summonerId varchar(255), puuid varchar(255), leaguePoints int, `rank` varchar(255), wins int, losses int)"
cursor.execute(query)
challenger = pd.read_csv('challenger_user.csv')
for index, row in challenger.iterrows():
    query = "insert into challenger_user (summonerId, puuid, leaguePoints, `rank`, wins, losses) values (%s, %s, %s, %s, %s, %s)"
    cursor.execute(query, (row['summonerId'], row['puuid'], row['leaguePoints'], row['rank'], row['wins'], row['losses']))
conn.commit()
conn.close()

In [21]:
query = "create table if not exist challenger_matchID (puuid varchar(255), match_ids list)"
cursor.execute(query)
challenger_matchID = pd.read_csv('challenger_match_ids.csv')
for index, row in challenger_matchID.iterrows():
    query = "insert into challenger_matchID (puuid, matchID) values (%s, %s)"
    cursor.execute(query, (row['puuid'], row['matchID']))
conn.commit()
cursor.close()
conn.close()

InterfaceError: (0, '')